In [1]:
from pathlib import Path
import os
import slicer

data_dir = Path("/data")
stl_dir = data_dir / "masks" / "STL"
img_dir = data_dir / "images" / "nifti"
out_dir = data_dir / "masks" / "nifti"
out_dir.mkdir(exist_ok=True)

In [2]:
# taken from https://discourse.slicer.org/t/converting-stl-files-to-binary-label-maps-in-nii-format-using-python/13038/9
def stl_to_nifti(stl_path, output_path, reference_volume_path):
    referenceVolumeNode = slicer.util.loadVolume(reference_volume_path)
    segmentationNode = slicer.util.loadSegmentation(stl_path)
    outputLabelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(segmentationNode, outputLabelmapVolumeNode, referenceVolumeNode)
    slicer.util.saveNode(outputLabelmapVolumeNode, output_path)

In [ ]:
for i, fname in enumerate(os.listdir(str(stl_dir))[1590:]):
    print(i, fname)
    stl_path = stl_dir / fname
    fname_split = fname[:-4].split("-")
    patient_ID = ('-').join(fname_split[:5])
    series_UID = fname_split[-1]
    ROI = fname_split[5]
    ref_img_path = img_dir / patient_ID / (series_UID + '.nii.gz')
    if ref_img_path.exists():
        (out_dir / patient_ID).mkdir(exist_ok=True)
        output_path = out_dir / patient_ID / (f"{ROI}_{series_UID}.nii.gz")
        if not output_path.exists():
            stl_to_nifti(str(stl_path), str(output_path), str(ref_img_path))